This example makes a reactor geometry and a neutronics model. A homogenised material made of enriched lithium lead and eurofer is being used as the blanket material for this simulation in order to demonstrate the use of more complex materials.

In [ ]:
import neutronics_material_maker as nmm
import openmc
import paramak


# makes the 3d geometry
my_reactor = paramak.BallReactor(
    inner_bore_radial_thickness=10,
    inboard_tf_leg_radial_thickness=30,
    center_column_shield_radial_thickness=60,
    divertor_radial_thickness=150,
    inner_plasma_gap_radial_thickness=30,
    plasma_radial_thickness=300,
    outer_plasma_gap_radial_thickness=30,
    firstwall_radial_thickness=30,
    blanket_radial_thickness=50,
    blanket_rear_wall_radial_thickness=30,
    elongation=2,
    triangularity=0.55,
    outboard_tf_coil_radial_thickness=100,
    outboard_tf_coil_poloidal_thickness=50,
    rotation_angle=90,
)

# method is set to Trelis or Cubit by default to avoid overlaps in the geometry
# pymoab is used as it is open source and can be tested in the CI
# if you have Trelis or Cubit then this line can be deleted
my_reactor.method='pymoab'

my_reactor.show()

In [ ]:
source = openmc.Source()
radius = openmc.stats.Discrete([300], [1])  # at the major radius
z_values = openmc.stats.Discrete([0], [1])  # middle of the plasma (vertically)
angle = openmc.stats.Uniform(a=0., b=2 * 3.14159265359)  # 360 degrees
source.space = openmc.stats.CylindricalIndependent(
    r=radius,
    phi=angle,
    z=z_values,
    origin=(0.0, 0.0, 0.0)
)
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([14_060_000], [1])

In [ ]:
# makes the neutronics material
neutronics_model = paramak.NeutronicsModel(
    geometry=my_reactor,
    source=source,
    materials={
        'inboard_tf_coils_mat': 'copper',
        'center_column_shield_mat': 'WC',
        'divertor_mat': 'eurofer',
        'firstwall_mat': 'eurofer',
        'blanket_mat': 'eurofer',
        'blanket_rear_wall_mat': 'eurofer'},
)

In [ ]:
# exports the geometry and source in 2d (RZ) viewplane where R stands for
# radius
neutronics_model.export_html(
    filename='2d_source.html',
    view_plane='RZ',
    number_of_source_particles=100
)

In [ ]:
# exports the geometry and source in 3d (XYZ) viewplane
neutronics_model.export_html(
    filename='3d_source.html',
    view_plane='XYZ',
    number_of_source_particles=100
)